# **Homework 1. Programming in Racket**
## Evgeny Bobkunov e.bobkunov@innopolis.university SD-03

## **1.1 Pseudo-Random Generation**

In this section, you will implement a pseudo-random number generator using the linear congruential
method.

**Exercise 1.1.1** (3 points). Implement the following functions which will serve as components of a
linear congruential generator:

1. `(mk-gen n)` — creating a new (state of a) generator from a seed integer value `n`

    ```
    (mk-gen 123)
    ; #<gen>
    ```

2. `(next-gen g)` — given a (state of a) generator, produce the next (state) using the linear congruential formula:

   $G_{n+1} = (aG_n + c)$ mod $m$

You may use any valid constants *a*, *c*, and *m* which are consistent with a pseudo-random number
generation.

```
(next-gen (mk-gen 123))
; #<gen>
```

In [11]:

(define a 48271)
(define c 125)
(define m 2147483647)

(define (mk-gen n)
  n)

(define (next-gen g)
  (modulo (+ (* a g) c) m))

(define g (mk-gen 123))

(printf "Initial generator state: ~a\n" g)
(define g1 (next-gen g))
(printf "First generated number: ~a\n" g1)

(define g2 (next-gen g1))
(printf "Second generated number: ~a\n" g2)

(define g3 (next-gen g2))
(printf "Third generated number: ~a\n" g3)

(define g4 (next-gen g3))
(printf "Fourth generated number: ~a\n" g4)

(define g5 (next-gen g4))
(printf "Fifth generated number: ~a\n" g5)


Initial generator state: 123
First generated number: 5937458
Second generated number: 991710192
Third generated number: 1284702880
Fourth generated number: 1007446186
Fifth generated number: 667658216


**Exercise 1.1.2** (4 points). Implement a function `gen-integer` such that `((gen-integer i j) g)`
generates a random integer in the range from *i* to *j* (not including *j*), with a given generator (state)
*g*.

```
((gen-integer 10 34) (mk-gen 123))
; '(13 . #<gen>)
((gen-integer 10 34) (mk-gen 123))
; '(13 . #<gen>) must be the same as above!
```

```
((gen-integer 10 34) (next-gen (mk-gen 123)))
; '(10 . #<gen>)
((gen-integer 10 34) (cdr ((gen-integer 10 34) (mk-gen 123))))
; '(10 . #<gen>) must be the same as above!
```

Note that `gen-integer` takes 2 arguments and produces a function that takes in the generator
(state)! This is by design, to simplify later combinators (functions that combine several generators).
Indeed, we will expect most combinators to be (or produce) functions that take in some generator
(state) *g* and produce a pair `(x . g2)` where *x* is a (pseudo)random value and *g2* is the new generator
(state).

In [13]:

(define a 48271)
(define c 125)
(define m 2147483647)

(define (mk-gen n)
  n)

(define (next-gen g)
  (modulo (+ (* a g) c) m))

(define (gen-integer i j)
  (lambda (g)
    (let* ((range (- j i))
           (random-number (next-gen g))
           (scaled-number (+ i (modulo random-number range))))
      (cons scaled-number (next-gen g)))))

(define g (mk-gen 123))
(printf "Generating random integers between 10 and 34:\n")

(define gen-fn (gen-integer 10 34))

(printf "First call: ~a\n" (gen-fn g))
(printf "Second call: ~a\n" (gen-fn g))
(printf "Next state: ~a\n" (next-gen g))


Generating random integers between 10 and 34:
First call: (12 . 5937458)
Second call: (12 . 5937458)
Next state: 5937458


**Exercise 1.1.3** (4 points). Implement a function `stream-integers` such that `(stream-integers n m g)`
produces an infinite stream of random integers in the range from *n* to *m*, starting from a given generator
(state) *g*:

```
(stream->list (stream-take (stream-integers 1 10 (mk-gen 123)) 5))
; '(7 7 6 9 3)
(stream->list (stream-take (stream-integers 1 10 (mk-gen 123)) 10))
; '(7 7 6 9 3 1 3 2 4 7) the first 5 integers should be the same as above!
```

In [15]:
(require racket/stream)

(define a 48271)
(define c 125)
(define m 2147483647)

(define (mk-gen n)
  n)

(define (next-gen g)
  (modulo (+ (* a g) c) m))

(define (gen-integer i j g)
  (let* ((range (+ 1 (- j i)))
         (scaled-number (+ i (modulo g range))))
    scaled-number))

(define (stream-integers n m g)
  (stream-cons
   (gen-integer n m g)
   (stream-integers n m (next-gen g))))

(define g (mk-gen 123))

(define s1 (stream-take (stream-integers 1 10 g) 5))
(printf "First 5 random integers: ~a\n" (stream->list s1))

(define s2 (stream-take (stream-integers 1 10 g) 10))
(printf "First 10 random integers: ~a\n" (stream->list s2))

First 5 random integers: (4 9 3 1 7)
First 10 random integers: (4 9 3 1 7 7 3 4 1 3)


**Exercise 1.1.4** (3 points). Implement a higher-order function `stream-random` such that `(stream-random f g)`
produces an infinite stream of random values, created by a function *f* starting from a given generator
(state) *g*. The function *f* is expected to take a generator (state) and produce a pair of a random value
and a new generator state.

```
(stream->list (stream-take (stream-random (gen-integer 1 10) (mk-gen 123)) 5))
; '(7 7 6 9 3) must be the same as stream-integers example above
(stream->list (stream-take (stream-random (gen-integer 1 10) (mk-gen 123)) 10))
; '(7 7 6 9 3 1 3 2 4 7) the first 5 integers should be the same as above!
```

In [17]:

(define a 48271)
(define c 125)
(define m 2147483647)

(define (mk-gen n)
  n)

(define (next-gen g)
  (modulo (+ (* a g) c) m))

(define (gen-integer i j)
  (lambda (g)
    (let* ((range (+ 1 (- j i)))
           (scaled-number (+ i (modulo g range))))
      (cons scaled-number (next-gen g)))))

(define (stream-random f g)
  (let* ((result (f g))
         (value (car result))
         (new-g (cdr result)))
    (stream-cons value (stream-random f new-g))))

(define g (mk-gen 123))

(define s1 (stream-take (stream-random (gen-integer 1 10) g) 5))
(printf "First 5 random integers: ~a\n" (stream->list s1))

(define g (mk-gen 123))

(define s2 (stream-take (stream-random (gen-integer 1 10) g) 10))
(printf "First 10 random integers: ~a\n" (stream->list s2))

First 5 random integers: (4 9 3 1 7)
First 10 random integers: (4 9 3 1 7 7 3 4 1 3)


## **1.2 Generating User-Defined Structures**

**Exercise 1.2.1** (3 points). Implement a function `gen-bool` such that `(gen-bool g)` produces a
random boolean value given a generator (state) *g*:

```
(stream->list (stream-take (stream-random gen-bool (mk-gen 13123)) 5))
; '(#f #t #f #f #f)
```

In [19]:

(define a 48271)
(define c 125)
(define m 2147483647)

(define (mk-gen n)
  n)

(define (next-gen g)
  (modulo (+ (* a g) c) m))

(define (gen-bool g)
  (let* ((random-num (next-gen g))
         (bool-value (even? random-num)))
    (cons bool-value (next-gen g))))

(define (stream-random f g)
  (let* ((result (f g))
         (value (car result))
         (new-g (cdr result)))
    (stream-cons value (stream-random f new-g))))

(define g (mk-gen 13123))
(define s (stream-take (stream-random gen-bool g) 5))
(printf "First 5 random boolean values: ~a\n" (stream->list s))

First 5 random boolean values: (#t #f #t #f #f)


**Exercise 1.2.2** (3 points). Implement a function `gen-element` such that `((gen-element lst) g)`
produces a random element of a list *lst* given a generator (state) *g*:

```
(stream->list (stream-take (stream-random (gen-element '(1 2 3)) (mk-gen 13)) 5))
; '(2 1 1 2 3)
```

In [23]:

(define a 1664525)
(define c 1013904223)
(define m (expt 2 32))

(define (mk-gen n)
  n)

(define (next-gen g)
  (modulo (+ (* a g) c) m))

(define (gen-element lst)
  (lambda (g)
    (let* ((len (length lst))
           (index (modulo g len))
           (element (list-ref lst index)))
      (cons element (next-gen g)))))

(define (stream-random f g)
  (let* ((result (f g))
         (value (car result))
         (new-g (cdr result)))
    (stream-cons value (stream-random f new-g))))

(define g (mk-gen 13))

(define s (stream-take (stream-random (gen-element '(1 2 3)) g) 5))
(printf "First 5 random elements: ~a\n" (stream->list s))

First 5 random elements: (2 1 3 1 1)


**Exercise 1.2.3** (5 points). Implement a higher-order function `gen-pair` such that `((gen-pair f1 f2) g)`
produces a random pair with first element produced by the function *f1* and the second function produced by the function *f2*, given a generator (state) *g*. Note that the new generator, produced by *f1* should be passed to *f2*.

```
(stream->list (stream-take (stream-random (gen-pair gen-bool gen-bool) (mk-gen 31337)) 3))
; '((#f . #t) (#t . #f) (#t . #t))
```

In [25]:

(define a 48271)
(define c 125)
(define m 2147483647)

(define (mk-gen n)
  n)

(define (next-gen g)
  (modulo (+ (* a g) c) m))

(define (gen-bool g)
  (let* ((random-number (next-gen g))             
         (bool-value (if (zero? (modulo random-number 2)) #f #t)))  
    (begin
      (cons bool-value random-number))))

(define (gen-pair f1 f2)
  (lambda (g)
    (let* ((result1 (f1 g))                       
           (v1 (car result1))                     
           (new-gen (cdr result1))                
           (result2 (f2 new-gen))                 
           (v2 (car result2))                     
           (final-gen (cdr result2)))             
      (cons (cons v1 v2) final-gen))))            

(define (stream-random f g)
  (let* ((result (f g))
         (value (car result))
         (new-gen (cdr result)))
    (stream-cons value (stream-random f new-gen))))

(define g (mk-gen 31337))

(define s1 (stream-take (stream-random (gen-pair gen-bool gen-bool) g) 3))

(printf "First 3 random pairs using gen-pair: ~a\n" (stream->list s1))

First 3 random pairs using gen-pair: ((#f . #f) (#t . #t) (#f . #t))


**Exercise 1.2.4** (5 points). Implement a higher-order function `gen-list` such that `((gen-list f n) g)` produces a random list of *n* elements, each of which is produced by the function *f* given a generator
(state) *g*. Note that the new generator, produced by *f* to generate *i*th element should be then passed
to generate (*i* + 1)th element.

```
((gen-list (gen-integer 1 10) 10) (mk-gen 123))
; '((7 7 6 9 3 1 3 2 4 7) . #<gen>)
```

In [27]:

(define a 48271)
(define c 125)
(define m 2147483647)

(define (mk-gen n)
  n)

(define (next-gen g)
  (modulo (+ (* a g) c) m))

(define (gen-integer min max)
  (lambda (g)
    (let* ((random-number (next-gen g))
           (bounded-number (+ min (modulo random-number (+ 1 (- max min))))))
      (cons bounded-number (next-gen g)))))

(define (gen-bool g)
  (let* ((random-number (next-gen g))             
         (bool-value (if (zero? (modulo random-number 2)) #f #t)))  
    (begin
      (cons bool-value random-number))))

(define (gen-pair f1 f2)
  (lambda (g)
    (let* ((result1 (f1 g))                       
           (v1 (car result1))                     
           (new-gen (cdr result1))                
           (result2 (f2 new-gen))                 
           (v2 (car result2))                     
           (final-gen (cdr result2)))             
      (cons (cons v1 v2) final-gen))))            

(define (stream-random f g)
  (let* ((result (f g))
         (value (car result))
         (new-gen (cdr result)))
    (stream-cons value (stream-random f new-gen))))

(define (count-outcomes pairs)
  (let loop ((pairs pairs) (both-true 0) (both-false 0) (one-true 0))
    (if (null? pairs)
        (list both-true both-false one-true)
        (let* ((pair (car pairs))
               (first (car pair))
               (second (cdr pair)))
          (cond
            ((and first second) (loop (cdr pairs) (+ both-true 1) both-false one-true))
            ((and (not first) (not second)) (loop (cdr pairs) both-false (+ both-false 1) one-true))
            (else (loop (cdr pairs) both-true both-false (+ one-true 1))))))))

(define (gen-list f n)
  (lambda (g)
    (define (loop i g)
      (if (= i 0)
          (cons '() g)
          (let* ((result (f g))
                 (value (car result))
                 (new-gen (cdr result))
                 (rest (loop (- i 1) new-gen)))
            (cons (cons value (car rest)) (cdr rest)))))
    (loop n g)))

(define g (mk-gen 123))
(define generated-list ((gen-list (gen-integer 1 10) 10) g))

(printf "Generated list: ~a\n" (car generated-list))
(printf "Final generator state: ~a\n" (cdr generated-list))


Generated list: (9 3 1 7 7 3 4 1 3 4)
Final generator state: 851185703


**Exercise 1.2.5** (5 points). Implement a polyvariadic higher-order function `gen*` such that `((gen* f1 ... fn) g)`
produces a list of *n* elements where *i*th element is produced by a function *fi*. Note that the new generator, produced by *fi* to generate *i*th element should be then passed to generate (*i* + 1)th element.

```
((gen*) (mk-gen 123))
; '(() . #<gen>)
((gen* (gen-integer 1 10)) (mk-gen 123))
; '((7) . #<gen>)
((gen* (gen-integer 1 10) (gen-integer 10 20)) (mk-gen 123))
; '((7 16) . #<gen>)
```

To define a polyvariadic function, follow the following definition pattern, which binds a list of all
arguments of `gen*` in the variable *args*:

```
(define (gen* . args) ...)
```

In [1]:

(define a 48271)
(define c 125)
(define m 2147483647)

(define (mk-gen n)
  n)


(define (next-gen g)
  (modulo (+ (* a g) c) m))


(define (gen-integer min max)
  (lambda (g)
    (let* ((random-number (next-gen g))
           (bounded-number (+ min (modulo random-number (+ 1 (- max min))))))
      (cons bounded-number (next-gen g)))))

(define (gen-bool g)
  (let* ((random-number (next-gen g))             
         (bool-value (if (zero? (modulo random-number 2)) #f #t)))  
    (begin
      (cons bool-value random-number))))

(define (gen-pair f1 f2)
  (lambda (g)
    (let* ((result1 (f1 g))                       
           (v1 (car result1))                     
           (new-gen (cdr result1))                
           (result2 (f2 new-gen))                 
           (v2 (car result2))                     
           (final-gen (cdr result2)))             
      (cons (cons v1 v2) final-gen))))            


(define (stream-random f g)
  (let* ((result (f g))
         (value (car result))
         (new-gen (cdr result))) 
    (stream-cons value (stream-random f new-gen))))

(define (count-outcomes pairs)
  (let loop ((pairs pairs) (both-true 0) (both-false 0) (one-true 0))
    (if (null? pairs)
        (list both-true both-false one-true)
        (let* ((pair (car pairs))
               (first (car pair))
               (second (cdr pair)))
          (cond
            ((and first second) (loop (cdr pairs) (+ both-true 1) both-false one-true)) 
            ((and (not first) (not second)) (loop (cdr pairs) both-false (+ both-false 1) one-true))
            (else (loop (cdr pairs) both-true both-false (+ one-true 1))))))))

(define (gen* . args)
  (lambda (g)
    (define (loop args g)
      (if (null? args)
          (cons '() g)
          (let* ((f (car args))
                 (result (f g))
                 (value (car result))
                 (new-gen (cdr result))
                 (rest (loop (cdr args) new-gen)))
            (cons (cons value (car rest)) (cdr rest)))))
    (loop args g)))

(define g (mk-gen 123))

(printf "No generators: ~a\n" ((gen*) g))
(printf "Single generator: ~a\n" ((gen* (gen-integer 1 10)) g))
(printf "Multiple generators: ~a\n" ((gen* (gen-integer 1 10) (gen-integer 10 20)) g))


No generators: (() . 123)
Single generator: ((9) . 5937458)
Multiple generators: ((9 10) . 991710192)


**Exercise 1.2.6** (5 points). Implement a polyvariadic higher-order function `gen-struct` such that
`((gen-struct mk f1 ... fn) g)` produces a random (user-defined) structure by applying *mk* to the
*n* arguments where *i*th argument is produced by a function `fi`. Note that the new generator, produced
by `fi` to generate *i*th argument should be then passed to generate (*i* + 1)th argument.

```
(struct person (name age))

(let [(p ((gen-struct person
                        (gen-element '(Anna Boris Charlie))
                        (gen-integer 20 30))
            (mk-gen 123)))]
    (displayln (person-name (car p)))
    (displayln (person-age (car p)))
    (displayln (cdr p)))
; Anna
; 26
; #<gen>
```

In [24]:

(define a 48271)
(define c 125)
(define m 2147483647)

(define (mk-gen n)
  n)

(define (next-gen g)
  (modulo (+ (* a g) c) m))

(define (gen-integer min max)
  (lambda (g)
    (let* ((random-number (next-gen g))
           (bounded-number (+ min (modulo random-number (+ 1 (- max min))))))
      (cons bounded-number (next-gen g)))))

(define (gen-bool g)
  (let* ((random-number (next-gen g))             
         (bool-value (if (zero? (modulo random-number 2)) #f #t)))  
    (cons bool-value (next-gen g))))

(define (gen-element lst)
  (lambda (g)
    (let* ((len (length lst))
           (index (modulo (next-gen g) len)))
      (cons (list-ref lst index) (next-gen g)))))

(define (gen-struct mk . args)
  (lambda (g)
    (define (loop args g acc)
      (if (null? args)
          (let ((struct (apply mk (reverse acc))))
            (cons struct g))
          (let* ((f (car args))
                 (result (f g))
                 (value (car result))
                 (new-gen (cdr result))) 
            (loop (cdr args) new-gen (cons value acc)))))
    (loop args g '())))  

(struct person (name age))

(let* ((gen (mk-gen 123))
       (result ((gen-struct person
                   (gen-element '(Anna Boris Charlie))
                   (gen-integer 20 30))
                 gen)))
  (define p (car result))
  (define new-gen (cdr result))
  (displayln (person-name p))
  (displayln (person-age p))
  (displayln new-gen))


Charlie
20
991710192


**Exercise 1.2.7** (+0.25% extra credit). Define `gen-int->int` such that `(gen-int->int g)` produces a
random *function* from integers to integers. The generated functions must at least include a possibility
of an identity function (λn.n), a constant function (λn.m, where m is some (random) integer), an
addition of another (random) integer (λn.n + m), and a multiplication by another (random) integer
(λn.n · m).

```
(map (lambda (f) (f 5))
    (stream->list (stream-take (stream-random gen-int->int (mk-gen 4)) 10)))
; '(-91 103 57 35 -83 31 -95 83 53 63)
```

In [64]:

(define a 48271)
(define c 125)
(define m 2147483647)

(define (mk-gen n)
  n)

(define (next-gen g)
  (modulo (+ (* a g) c) m))

(define (gen-range-int min max)
  (lambda (g)
    (let* ((range (+ 1 (- max min)))
           (random-num (modulo g range))
           (result (+ min random-num)))
      (cons result (next-gen g)))))

(define (gen-int->int g)
  (let* ((choice-result ((gen-range-int 0 3) g))
         (choice (car choice-result))
         (g2 (cdr choice-result)))
    (case choice
      ((0) ; Identity function
       (cons (lambda (n) n) g2))
      ((1) ; Constant function
       (let* ((const-result ((gen-range-int -100 100) g2))
              (const-value (car const-result))
              (g3 (cdr const-result)))
         (cons (lambda (n) const-value) g3)))
      ((2) ; Addition function
       (let* ((add-result ((gen-range-int -100 100) g2))
              (add-value (car add-result))
              (g3 (cdr add-result)))
         (cons (lambda (n) (+ n add-value)) g3)))
      ((3) ; Multiplication function
       (let* ((mul-result ((gen-range-int -10 10) g2))
              (mul-value (car mul-result))
              (g3 (cdr mul-result)))
         (cons (lambda (n) (* n mul-value)) g3))))))

(define (stream-random f g)
  (let* ((result (f g))
         (value (car result))
         (new-g (cdr result)))
    (stream-cons value (stream-random f new-g))))

(define g (mk-gen 4))
(define s (stream-take (stream-random gen-int->int g) 10))
(define results (map (lambda (f) (f 5)) (stream->list s)))
(printf "Results of applying 10 random functions to 5: ~a\n" results)


Results of applying 10 random functions to 5: (5 -90 -20 52 93 5 13 -25 7 -40)


## **1.3 Checking User-Defined Properties**

In this section, you will implement a checker for user-defined properties, enabling randomized propertybased testing. A property will be defined by the following structure:

```
(struct property (generator predicate))
```

**Exercise 1.3.1** (5 points). Implement a function `stream-counterexamples` such that
`(stream-counterexamples property g)` produces a stream of (randomly-generated) counterexamples for a given property `property`. To prevent infinite search, limit the search space to 100 random
inputs.

```
(stream->list (stream-counterexamples
                (property (gen-integer 1 100)
                    (lambda (x) (> x 5)))
                (mk-gen 123)))
; '(4 4 1)

(stream->list (stream-counterexamples
                (property (gen-integer 1 100)
                    (lambda (x) (> x 0)))
                (mk-gen 123)))
; '()
```

In [92]:

(define a 48271)
(define c 125)
(define m 2147483647)

(define (mk-gen n)
  n)

(define (next-gen g)
  (modulo (+ (* a g) c) m))

(struct property (generator predicate))

(define (gen-integer i j)
  (lambda (g)
    (let* ((range (+ 1 (- j i)))
           (random-num (modulo g range))
           (result (+ i random-num)))
      (cons result (next-gen g)))))

(define (stream-counterexamples prop g)
  (define (generate-counterexamples count g)
    (if (= count 100)
        empty-stream
        (let* ((gen-result ((property-generator prop) g))
               (value (car gen-result))
               (new-g (cdr gen-result)))
          (if ((property-predicate prop) value)
              (generate-counterexamples (+ count 1) new-g)
              (stream-cons value (generate-counterexamples (+ count 1) new-g))))))
  (generate-counterexamples 0 g))


(define prop1 (property (gen-integer 1 100) (lambda (x) (> x 5))))
(define prop2 (property (gen-integer 1 100) (lambda (x) (> x 0))))

(define g (mk-gen 123))

(printf "Counterexamples for prop1: ~a\n" 
        (stream->list (stream-counterexamples prop1 g)))

(printf "Counterexamples for prop2: ~a\n" 
        (stream->list (stream-counterexamples prop2 g)))


Counterexamples for prop1: (4 5 1)
Counterexamples for prop2: ()


**Exercise 1.3.2** (3 points). Implement a function `stream-counterexample` such that
`(stream-counterexample property g)` produces a stream of at most one (randomly-generated) counterexample for a given property `property`. To prevent infinite search, limit the search space to 100
random inputs.

```
(stream->list (stream-counterexample
                (property (gen-integer 1 100)
                    (lambda (x) (> x 5)))
                (mk-gen 123)))
; '(4)

(stream->list (stream-counterexample
                (property (gen-integer 1 100)
                    (lambda (x) (> x 0)))
                (mk-gen 123)))
; '()
```

In [97]:

(define a 48271)
(define c 125)
(define m 2147483647)

(define (mk-gen n)
  n)

(define (next-gen g)
  (modulo (+ (* a g) c) m))

(struct property (generator predicate))

(define (gen-integer i j)
  (lambda (g)
    (let* ((range (+ 1 (- j i)))
           (random-num (modulo g range))
           (result (+ i random-num)))
      (cons result (next-gen g)))))

(define (stream-counterexample prop g)
  (define (find-counterexample count g)
    (if (= count 100)
        empty-stream
        (let* ((gen-result ((property-generator prop) g))
               (value (car gen-result))
               (new-g (cdr gen-result)))
          (if ((property-predicate prop) value)
              (find-counterexample (+ count 1) new-g)
              (stream value)))))
  (find-counterexample 0 g))

(define prop1 (property (gen-integer 1 100) (lambda (x) (> x 5))))
(define prop2 (property (gen-integer 1 100) (lambda (x) (> x 0))))

(define g (mk-gen 123))

(printf "Counterexample for prop1: ~a\n" 
        (stream->list (stream-counterexample prop1 g)))

(printf "Counterexample for prop2: ~a\n" 
        (stream->list (stream-counterexample prop2 g)))


Counterexample for prop1: (4)
Counterexample for prop2: ()


**Exercise 1.3.3** (5 points). Implement a function `check` such that `(check property)` checks if a
property is true on 100 randomly generated values. You may use a non-pure procedure
to generate a
random seed. If case of success, display `"Success"`, otherwise, display a counterexample.

```
; for all 1 <= n < 10
; we have
; n < 11
(check (property (gen-integer 1 10) (lambda (n) (< n 11))))
; Success
```

```
; for all 1 <= n < 10
; we have
; n < 5
(check (property (gen-integer 1 10) (lambda (n) (< n 5))))
; Falsified!
; Counterexample: 7
```

```
; for all 1 <= x < 10
; and 11 <= y < 20
; we have
; x < y < (x + y)
(check (property
    (gen-pair (gen-integer 1 10) (gen-integer 11 20))
    (lambda (xy)
        (let [(x (car xy))
            (y (cdr xy))]
        (< x y (+ x y))))))
; Success

; Falsified!
; Counterexample: 7

; for all 1 <= x < 10
; and 11 <= y < 20
; we have
; x < (y - x)
(check (property
    (gen-pair (gen-integer 1 10) (gen-integer 11 20))
    (lambda (xy)
        (let [(x (car xy))
            (y (cdr xy))]
        (< x (- y x))))))
; Falsified!
; Counterexample: (7 . 12)
```

In [104]:

(define a 48271)
(define c 125)
(define m 2147483647)

(define (mk-gen n)
  n)

(define (next-gen g)
  (modulo (+ (* a g) c) m))

(struct property (generator predicate))

(define (gen-integer i j)
  (lambda (g)
    (let* ((range (+ 1 (- j i)))
           (random-num (modulo g range))
           (result (+ i random-num)))
      (cons result (next-gen g)))))

(define (gen-pair f1 f2)
  (lambda (g)
    (let* ((result1 (f1 g))
           (value1 (car result1))
           (g2 (cdr result1))
           (result2 (f2 g2))
           (value2 (car result2))
           (g3 (cdr result2)))
      (cons (cons value1 value2) g3))))

(define (check prop)
  (define (check-property count g)
    (if (= count 100)
        (display "Success\n")
        (let* ((gen-result ((property-generator prop) g))
               (value (car gen-result))
               (new-g (cdr gen-result)))
          (if ((property-predicate prop) value)
              (check-property (+ count 1) new-g)
              (begin
                (display "Falsified!\n")
                (display "Counterexample: ")
                (display value)
                (newline))))))
  
  (check-property 0 (mk-gen (random 1000000))))

(define prop1 (property (gen-integer 1 10) (lambda (n) (< n 11))))
(define prop2 (property (gen-integer 1 10) (lambda (n) (< n 5))))
(define prop3 (property
                (gen-pair (gen-integer 1 10) (gen-integer 11 20))
                (lambda (xy)
                  (let ((x (car xy))
                        (y (cdr xy)))
                    (< x y (+ x y))))))
(define prop4 (property
                (gen-pair (gen-integer 1 10) (gen-integer 11 20))
                (lambda (xy)
                  (let ((x (car xy))
                        (y (cdr xy)))
                    (< x (- y x))))))

(printf "Checking prop1:\n")
(check prop1)

(printf "\nChecking prop2:\n")
(check prop2)

(printf "\nChecking prop3:\n")
(check prop3)

(printf "\nChecking prop4:\n")
(check prop4)

Checking prop1:
Success

Checking prop2:
Falsified!
Counterexample: 7

Checking prop3:
Success

Checking prop4:
Falsified!
Counterexample: (8 . 12)


## **1.4 Implementing Properties**

**Exercise 1.4.1** (3 points). Implement a function `random-2D-point` to generate random 2D points (it
is enough to generate points with integer coordinates):

```
(stream->list (stream-take (stream-random gen-2D-point (mk-gen 123)) 5))
; '((-7 . 6) (-5 . -8) (1 . 2) (7 . 0) (9 . 2))
```

In [20]:

(define (random-in-range low high)
  (+ low (random (+ 1 (- high low)))))

(define (gen-2D-point)
  (cons (random-in-range -10 10) (random-in-range -10 10)))

(define (stream-random gen)
  (cons (gen) (delay (stream-random gen))))

(define (stream-take s n)
  (if (or (zero? n) (null? s))
      '()
      (cons (car s) (stream-take (force (cdr s)) (sub1 n)))))

(define (stream->list s)
  (if (null? s)
      '()
      (cons (car s) (stream->list (force (cdr s))))))

(define random-points
  (stream->list (stream-take (stream-random gen-2D-point) 5)))

random-points


'((-9 . 6) (-2 . -2) (0 . -3) (1 . -10) (2 . 10))

**Exercise 1.4.2** (3 points). Implement a function `euclidean-distance-2D` to calculate Euclidean
distance between two 2D points:

```
(euclidean-distance-2D '(3 . 4) '(0 . 0))
; 5
```

Let's use the Euclidean distance formula:

$d = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}$

In [28]:

(define (euclidean-distance-2D point1 point2)
  (let* ((x1 (car point1))
         (y1 (cdr point1))
         (x2 (car point2))
         (y2 (cdr point2)))
    (sqrt (+ (expt (- x2 x1) 2)
              (expt (- y2 y1) 2)))))

(euclidean-distance-2D '(3 . 4) '(0 . 0))


5

**Exercise 1.4.3** (5 points). Implement a function `triangular` such that `(triangular gen mu)` produces a property that checks the triangular property for metric `µ` (mu) for three randomly generate
(via `gen`) values:

$∀p_1, p_2, p_3.µ(p1, p2) + µ(p2, p3) ≥ µ(p1, p3)$

```
(check (triangular gen-2D-point euclidean-distance-2D))
; Success

(define (bad-distance-2D p1 p2)
    (let [(x1 (car p1))
        (x2 (car p2))
        (y1 (cdr p1))
        (y2 (cdr p2))]
    (+ x1 x2 y1 y2)))

(check (triangular gen-2D-point bad-distance-2D))
; Falsified!
; Counterexample: ((-7 . 6) (-5 . -8) (1 . 2))
```

In [85]:

(define (gen-2D-point)
  (cons (random -10 10) (random -10 10)))


(define (euclidean-distance-2D p1 p2)
  (let* ([x1 (car p1)]
         [x2 (car p2)]
         [y1 (cdr p1)]
         [y2 (cdr p2)]
         [dx (- x1 x2)]
         [dy (- y1 y2)])
    (sqrt (+ (* dx dx) (* dy dy)))))

(define (bad-distance-2D p1 p2)
  (let* ([x1 (car p1)]
         [x2 (car p2)]
         [y1 (cdr p1)]
         [y2 (cdr p2)])
    (+ x1 x2 y1 y2)))

(define (triangular gen mu)
  (λ ()
    (let* ([p1 (gen)]
           [p2 (gen)]
           [p3 (gen)])
      (if (>= (+ (mu p1 p2) (mu p2 p3)) (mu p1 p3))
          #t
          (list p1 p2 p3)))))

(define (check prop)
  (let loop ([n 100])
    (cond
      [(zero? n) (displayln "Success")]
      [else
       (let ([result (prop)])
         (if result
             (if (eq? result #t)
                 (loop (sub1 n))
                 (begin
                   (displayln "Falsified!")
                   (displayln (format "Counterexample: ~a" result))))
             (loop (sub1 n))))])))

(check (triangular gen-2D-point euclidean-distance-2D))

(check (triangular gen-2D-point bad-distance-2D))


Success
Falsified!
Counterexample: ((-8 . 7) (-5 . -9) (7 . 8))


**Exercise 1.4.4** (5 points). Implement a function `functor-law-1` such that `(functor-law-1 map)` is
a property that checks if for any list `lst`, mapping an identity function does not modify it:

```
(map identity lst)
; should be the same as
lst
```

It is enough to check lists of integers of random size (up to 100 elements).

```
(check (functor-law-1 map))
; Success

(define (bad-map f xs) (reverse xs))
(check (functor-law-1 bad-map))
; Falsified!
; Counterexample: (6 5 2 1 2 7 0 9 2 9 8 9 0 5 6 5 4 7 6 7 2 7)
```

In [64]:

(define (range n)
  (if (<= n 0)
      '()
      (cons (- n 1) (range (- n 1)))))

(define (identity x)
  x)

(define (random-integer-list n)
  (map (lambda (_) (random -10 10)) (range n)))

(define (functor-law-1 map)
  (define (check-law lst)
    (let ([mapped (map identity lst)])
      (if (equal? mapped lst)
          #t
          (begin
            (printf "Counterexample: ~a\n" lst)
            #f))))
  (define random-size (random 1 101))
  (define lst (random-integer-list random-size))
  (check-law lst))

(define (check prop)
  (if prop
      (printf "Success\n")
      (printf "Failed!\n")))

(check (functor-law-1 map)) 

(define (bad-map f xs) (reverse xs))

(check (functor-law-1 bad-map))


Success
Counterexample: (-1 -2 0 5 5 0 1 4 5 -4 7 -1 1 -10 0 9 7 0 -10 5)
Failed!


**Exercise 1.4.5** (+0.25% extra credit). Implement a function `functor-law-2` such that `(functor-law-2 map)`
is a property that checks if for functions `f` and `g` and any list `lst`, mapping a composition of functions
is equivalent to a composition of maps:

```
(map f (map g lst))
; should be the same as
(map (lambda (x) (f (g x))) lst)
```

It is enough to check lists of integers of random size (up to 100 elements).

```
(check (functor-law-2 map))
; Success

(check (functor-law-2 bad-map))
; Falsified!
; Counterexample: ((#<procedure:...> #<procedure:...>) 2 1 2 7 0 9 2 9)
```

In [117]:

(define (functor-law-2 map)
  (λ ()
    (let* ([lst (build-list (random 101) (λ (_) (random 100)))]
           [f (λ (x) (+ x 1))]                            
           [g (λ (x) (* x 2))])                      
      (if (equal? (map f (map g lst))
                  (map (λ (x) (f (g x))) lst))
          #t
          (list f g lst)))))


(define (bad-map f lst)
  (if (null? lst)
      '()
      (cons (f (car lst)) lst)))

(define (check prop)
  (let loop ([n 100])
    (cond
      [(zero? n) (displayln "Success")]
      [else
       (let ([result (prop)])
         (if result
             (if (eq? result #t)
                 (loop (sub1 n))
                 (begin
                   (displayln "Falsified!")
                   (displayln (format "Counterexample: ~a" result))))
             (loop (sub1 n))))])))

(check (functor-law-2 map))

(check (functor-law-2 bad-map))


Success
Falsified!
Counterexample: (#<procedure:f> #<procedure:g> (81 19 60 76 99 14 57 11 53 74 28 94 41 38 99 40 80 30 56 34 36 0 18 65 53 66 30))


**Exercise 1.4.6** (+0.25% extra credit). Implement a function `filter-shortens-list` such that
`(filter-shortens-list filt)` is a property that checks `(filt p lst)` is shorter than `lst` for any
predicate `p` and any list `lst`. It is enough to check lists of integers of random size (up to 100 elements).

```
(filter p xs)
; should be the same length or shorter than xs
```

It is enough to check lists of integers of random size (up to 100 elements).

```
(check (filter-shortens-list filter))
; Success

(define (bad-filter p lst)
    (append (filter p lst) (filter p lst)))
(check (filter-shortens-list bad-filter))
Falsified!
Counterexample: (#<procedure:...> 2 1 2 7 0 9 2 9)
```

In [138]:

(define (filter-shortens-list filt)
  (λ ()
    (let* ([lst (build-list (random 101) (λ (_) (random 100)))] 
           [p (λ (x) (< x 50))]                                
           [filtered (filt p lst)])                            
      (if (<= (length filtered) (length lst))
          #t                                                  
          (list p lst)))))                                    


(define (bad-filter p lst)
  (append (filter p lst) (filter p lst)))

(define (check prop)
  (let loop ([n 100])
    (cond
      [(zero? n) (displayln "Success")]    
      [else
       (let ([result (prop)])       
         (if (eq? result #t)
             (loop (sub1 n))     
             (begin
               (displayln "Falsified!") 
               (displayln (format "Counterexample: ~a" result)))))]))) 

(check (filter-shortens-list filter))

(check (filter-shortens-list bad-filter))


Success
Falsified!
Counterexample: (#<procedure:p> (90 17 26 15 21 50 50 6 12 24 59))


## **1.5 Proving Properties**

In this section, using equational reasoning (aka the Substitution Model [1, §1.1.5]) prove formally the
properties that you have checked in the previous section.

**Exercise 1.5.1** (5 points). Prove the first functor law for `map` and lists. Specifically, provide a correct
definition of `map` and show that for any list `lst`, you have

```
(map identity lst) = ... = lst
```

1. Define `map`
   
    ```
    (define (map f lst)
      (if (null? lst)
        '()                         
        (cons (f (car lst))         
              (map f (cdr lst))))) 
    ```
    Base case is empty list. Then, apply `f` to the first element. After that recursively apply f to the rest of the list.

2. Prove the Functor Law

   1. Base Case:
      
      - For the empty list:

   
      ```
      map identity ’()=’()
      ```

      - This holds because by definition of `map`, when `lst` is empty, it returns an empty list.
   2. Inductive Step

      - Assume the property holds for a list `lst`, which means: `map identity lst=lst`
      - We want to prove it for the list `(cons x lst)` where `x` is the head of the list and `lst` is the tail.
        ```
        map identity (cons x lst)=cons (identity x) (map identity lst)
        ```
      - By the definition of the identity function: `identity x=x`
      - Therefore, we can substitute:
        ```
        map identity (cons x lst)=cons x (map identity lst)
        ```
      - By the inductive hypothesis: `map identity lst=lst`
      - We substitute this back into the equation:
        ```
          map identity (cons x lst)=cons x lst
        ```

Both the base case and inductive step hold true, therefore, we can conclude that for any list `lst`: `map identity lst=lst`

**Exercise 1.5.2** (5 points). Prove the second functor law for `map` and lists. Specifically, provide a
correct definition of `map` and show that for any *pure* functions `f` and `g` and any list `lst`, you have

```
(map f (map g lst)) = ... = (map (lambda (x) (f (g x))) lst)
```

1. Base Case (Empty List):
   
    When the list is empty `(lst = '())`, both sides of the equation should evaluate to an empty list:

    - Left-hand side:
      
        $\text{map } f (\text{map } g \, '()) = \text{map } f \, '() = '()$
      
      This holds because map applied to an empty list always returns an empty list.

    - Right-hand side:

        $\text{map } (\lambda (x) \, (f (g x))) \, '() = '()$
      
       This also holds because map applied to an empty list returns an empty list.

      **Thus, both sides are equal for the empty list.**

2. Inductive Step (Non-Empty List):

   Now, assume the property holds for a list `lst`. That is, assume:

   `map f(map g lst) = map (lambda(x)(f(gx)))lst`

   We need to prove the property holds for a list of the form `(cons x lst)`, where `x` is the head of the list and `lst` is the tail.

   - **LHS:**

     `map f(map g(cons x lst)) = map f(cons (gx)(map g lst))`

     By the definition of `map`, this becomes:

     `cons (f(gx))(map f(map g lst))`

     Using the inductive hypothesis, we know:

     `map f(map g lst) = map (lambda(x)(f(gx)))lst`

     Therefore, we substitute this in:

     `cons (f(gx))(map (lambda(x)(f(gx)))lst)`

   - **RHS:**
  
     `map (lambda(x)(f(gx)))(cons x lst) = cons ((lambda(x)(f(gx)))x)(map (lambda(x)(f(gx)))lst)`

     Simplifying the application of the lambda function:

     `cons (f(gx))(map (lambda(x)(f(gx)))lst)`

Thus, both the **LHS** and **RHS** simplify to the same expression:

`cons (f(gx))(map (lambda(x)(f(gx)))lst)`

Since both the base case (empty list) and the inductive step (non-empty list) have been shown to hold, we can conclude by mathematical induction that for all lists `lst` and for all functions `f` and `g`:

```
map f(map g lst)=map (lambda(x)(f(gx)))lst
```

**Exercise 1.5.3** (+0.25% extra credit). Prove that filtering a list cannot increase its length. Specifically, provide a correct definition of `filter` and show that for any *pure* predicate `p` and any list `lst`,
you have

```
(length (filter p lst)) <= (length lst)
```



1. Define `filter`
```
(define (filter p lst)
  (if (null? lst)
      '()                                    
      (let ([first (car lst)]                
            [rest (cdr lst)])                
        (if (p first)                        
            (cons first (filter p rest))     
            (filter p rest)))))
```

It applies a predicate `p` to each element of a list, retaining only those elements for which the predicate returns `#t`

2. Prove Using Induction (Empty list)
   
    1. Base Case:

       - When the list is empty `(lst = '())`:
       
       - Left-hand side:
    
        $\text{length}(\text{filter } p \, '()) = \text{length}('()) = 0$

       - Right-hand side (RHS):


        $\text{length}('()) = 0$

       Thus, for the empty list:

       $0≤0$

       This holds, so the property is true for the base case.

3. Inductive Step (Non-Empty List)

   Now, assume the property holds for a list `lst`, meaning:

   `length(filter p lst) ≤ length(lst)`

   Prove that the property holds for a list of the form `(cons x lst)`, where `x` is the head of the list and `lst` is the tail.

   Consider two cases for the element `x`:

   - **Case 1**: `p(x)` is true:
  
     In this case, `x` is included in the filtered list.

     The lengths of both sides of the equation are:

     **LHS**: `length(filter p(cons x lst)) = 1 + length(filter p lst)`

     **RHS**: `length(cons x lst) = 1 + length(lst)`

     By the inductive hypothesis: `length(filter p lst) ≤ length(lst)`

     Thus: `1 + length(filter p lst) ≤ 1 + length(lst)`

     Therefore: `length(filter p (cons x lst)) ≤ length(cons x lst)`

   - **Case 2** : `p(x)` is false:

     In this case, `x` is not included in the filtered list.

     The lengths of both sides of the equation are:

     **LHS**: `length(filter p (cons x lst)) = length(filter p lst)`

     **RHS**: `length(cons x lst) = 1 + length(lst)`

    By the inductive hypothesis: `length(filter p lst) ≤ length(lst)`

   Thus: `length(filter p (cons x lst)) ≤ 1 + length(lst)`

Since both the base case (empty list) and the inductive step (non-empty list) have been shown to hold, we can conclude by mathematical induction that for any predicate `p` and any list `lst`:

`length(filter plst) ≤ length(lst)`


This proves that filtering a list cannot increase its length.